**Ex. 2 DMRG implementation**

We start by creating a random MPS and right-normalizing it. We also need the MPO of the transverse field Ising Hamiltonian. These are two ingredients needed as an input to the actual DMRG.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import math
from scipy import linalg
import scipy.sparse
from scipy.sparse import linalg

from numpy import transpose as tr, conjugate as co
from scipy.linalg import expm, svd
from scipy.sparse.linalg import eigsh, LinearOperator
import math


Let us start with some utility functions to left and right nomalize the MPS at a site. This is the same as last week's exercise (Ex. 2) -- the code is given to you.

In [ ]:
def dot(A,B):
    """ Does the dot product like np.dot, but preserves the shapes also for singleton dimensions """
    s1 = A.shape
    s2 = B.shape
    return np.dot(A,B).reshape((s1[0],s2[1]))

def right_canonize(M1,M2,return_S = False):
    """ Right normalizes M2 into B matrix, M1 loses its canonization """
    s, da, db = M2.shape
    U, S, Vh = svd(M2.transpose((1,0,2)).reshape((da,s*db)))
    #this reshapes M2 and finds its svd
    B2     = Vh.reshape((Vh.shape[0],s,db)).transpose((1,0,2))[:,:da,:]
    M1     = np.tensordot(M1,dot(U[:,:min(da,np.shape(S)[0])],np.diag(S[:min(da,np.shape(S)[0])])),axes=((2),(0)))
    if return_S:
        return M1, B2, S
    else:
        return M1, B2

    
def left_canonize(M1,M2,return_S = False):
    """ Left normalizes M1 into A matrix, M2 loses its canonization"""
    s, da, db = M1.shape
    U, S, Vh = svd(M1.reshape((s*da,db)))
    A1      = U.reshape((s,da,U.shape[1]))[:,:,:min(db,np.shape(S)[0])]
    M2     = np.tensordot(dot(np.diag(S[:min(db,np.shape(S)[0])]),Vh[:min(db,np.shape(S)[0]),:]),M2,axes=((1),(1)))
    if return_S:
        return A1, M2, S
    else:
        return A1, M2


The MPO for the TFIM Hamiltonian is the same as in Ex. 2 of this week.

In [ ]:
sigma_x=np.array([[0,1],[1,0]])
sigma_y=np.array([[0+0j,0-1j],[0+0j,0+1j]])
sigma_z=np.array([[1,0],[0,-1]])

In [ ]:
def make_Ising(N, h=0):
    """ return MPO [O1, ...ON] of the transverse field Ising model with dim(Oi)=[s,s,da,db], here s=2 and da=db=3"""
    # implement the MPO representation of the transverse field Ising model here
    
    return O_Ising


We will now need functions that form right and left environments on the chain. Starting from the right of the chain, we contract indices up to some vertical cut and obtain $R^i_{a_i a'_i b_i}$ environment tensor that has two MPS virtual indices $a_i$ and $a'_i$ as well as one MPO index $b_i$. Note that we should work here with the right-normalized MPS representation. The recurrent formula for $R^{i + 1}_{a_{i + 1} a'_{i + 1} b_{i + 1}}$ is straightforward:
$$R^{i + 1}_{a_{i + 1} a'_{i + 1} b_{i + 1}} = \sum\limits_{\sigma_i \sigma'_i} \sum\limits_{b_i a_i a'_i} R^i_{a_i a'_i b_i} B^{a_{i + 1}}_{\sigma_i a_i} B^{\dagger a'_{i + 1}}_{\sigma'_i a'_i} W^{\sigma_i \sigma'_i}_{b_i b_{i + 1}}.$$

The same procedure applies to the construction of left environments $L^i_{a_i a'_i b_i}$, which are constructed from MPO $W_i$ and left-normalized MPS representation $A$.

In [ ]:
# ---- R^{i + 1}     -- B^{\dag} -- R^i
#      R^{i + 1}          |         R^i
# ---- R^{i + 1} === ---- W ------- R^i
#      R^{i + 1}          |         R^i
# ---- R^{i + 1}     ---- B ------- R^i

def add_site_to_R_env(R_env, B, W):
    """
    R_env: right environment from previous step; shape (da_psi, da_H, da_psi) 
    B: right-normalized; shape (s, db_psi, da_psi)
    W: MPO; shape (s, s, db_H, da_H)
    
    Returns
    R_env: updated right environment; shape (db_psi, db_H, db_psi)
    """
    
    # implement contractions (either np.tensordot or np.einsum)

    return R_env    

#  L^i -- A^{\dag} ---     L^{i + 1} ---
#  L^i       |             L^{i + 1}
#  L^i ------W ------- === L^{i + 1} ---
#  L^i       |             L^{i + 1}
#  L^i ------A -------     L^{i + 1} ---

def add_site_to_L_env(L_env, A, W):
    """
    L_env: left environment from previous step; shape (da_psi, da_H, da_psi) 
    A: left-normalized; shape (s, da_psi, db_psi)
    W: MPO; shape (s, s, da_H, db_H)
    
    Returns
    L_env: updated left environment; shape (db_psi, db_H, db_psi)
    """
        
    # implement contractions (either np.tensordot or np.einsum)

    return L_env2

In the DMRG algorithm, one locally solves the eigenvalue problem $$\sum\limits_{\sigma_l a_{l - 1} a_l} \left(\underbrace{\sum\limits_{b_l b_{l - 1}} L_{a_{l - 1} b_{l - 1} a'_{l - 1}} W^{\sigma'_l \sigma_l}_{b_{l - 1} b_l} R_{a_l b_l a'_l}}_{\hat H}\right) M^{\sigma_l}_{a_{l - 1} a_l} = \lambda M^{\sigma'_l}_{a'_{l - 1} a'_l}.$$

The underbraced expression is the local "Hamiltonian" acting on the MPS matrix at site $l$. The construction of this Hamiltonian itself is very inefficient, so one only defines the *action* of this Hamiltonian as a Linear Operator. (Recall this sort of procedure from Exercise Sheet 3, Question 1B.)

In [ ]:
def H_local(L_env, W, R_env, M):
    """
    L_env: left environment up to site l-l
    W: MPO at site l
    R_env: right environment from site l+1
    M: MPS matrix at site l
    """
    
    # distinguish between the boundary and the bulk
    s = 2    # dimension of the physical leg
    if len(M.shape) == 1:  # in case of the boundary
        flatten = True
        M = np.reshape(M, (s, L_env.shape[0], R_env.shape[0]))
    elif len(M.shape) == 3:  # in case of the bulk index
        flatten = False
    else:
        raise ValueError('Unknown format for M')
    
    # L -- (blank) --- R
    # L    |           R
    # L -- W --------- R
    # L    |           R
    # L -- M --------- R
    
    # contract L_env, M, W and R_env as shown above
    hpsi = ... # either np.tensordot or np.einsum
    
    if flatten:
        return hpsi.flatten()
    return hpsi

Now we are ready to run DMRG. For a given value of $h$ transverse field, we construct the MPO, create a random initial MPS, right-normalize it, precompute right environments.

Then the algorithm performs consequtive right and left "sweeps". Starting from the left, we compute local left environment, construct local "Hamiltonian", optimize MPS locally and normalize.


In [ ]:
def run_DMRG(h, L, chi = 60, nmax = 1000, verbose = False, atol=1e-12):
    """
    h: transverse field
    L: number of spins
    chi: maximum bond dimension
    nmax: maximum number of left and right sweeps
    
    returns: 
    energy: ground state energy
    Lambdas: list of singular value matrices
    """
    ### construct the MPO ###
    W = ...

    ### define a random MPS ###
    # we want a list of random MPS tensors from sites 0 to L-1
    # e.g. draw from a normal distribution (np.random.standard_normal)
    # shape of each tensor: (s, dright, dleft)
    # recall how the bond dimension grows from the left: 1, d, d^2...d^(N/2) and then decrease again d^(N/2),..., d^2, d
    # take care not to exceed the maximum bond dimension (chi)!
    
    M = [... for i in range(L)]
    
    ### right-normalize the MPS ###
    # start from the right edge: L-1, L-2...,1
    for i in range(L - 1, 0, -1):
        # implement that here
    # to run on the left boundary, introduce a fake matrix that we discard later
    _, M[0] = right_canonize(np.ones((1, 1, 1)), M[0])
    
    ### compute right environments ###
    # again start with a fake matrix on the right boundary
    R_environments = [np.ones((1, 1, 1))]    
    for i in range(1, L):
        # build the right environment from site 1...L-1
    
    
    ### now the left and right sweeps ###
    local_energy_previous = np.inf
    delta = np.inf
    
    for n in range(nmax):
        ### right sweep ###
        # start with a fake matrix on the left boundary
        L_environments = [np.ones((1, 1, 1))]  
    
        #Lambdas = []
        for i in range(L - 1):    # start from site 0, move one site to the right at each iteration
            # compute the local H dimension (to be used to construct the LinearOperator object)
            local_H_dim = ...
            
            # construct the LinearOperator object
            Hop = scipy.sparse.linalg.LinearOperator(...)
            
            # obtain best local MPS (linearized) and local energy using the Lanczos algorithm
            energy, V = scipy.sparse.linalg.eigsh(Hop, k = 1, v0 = M[i].flatten(), \
                                                  tol = 1e-2 if n < 2 else 0, which = 'SA')
            
            ### update step ###
            energy = energy[0]     # calculated ground state energy
            delta = energy - local_energy_previous
            local_energy_previous = energy 

            # reshape the obtained result to the shape of the MPS matrix
            M[i] = V.reshape(...)
            
            # left-normalize the result (it will affect M[i + 1], but we will optimize it in the next iteration)
            M[i], M[i + 1], Lambda = ...
            
            # add the left-normalized tensor to the left environment
            L_environments.append(...)

        ### repeat the same for left sweep ###
        R_environments = [np.ones((1, 1, 1))]
        
        Lambdas = []
        
        for i in range(L - 1, 0, -1):    # now start at site L-1, iterate until site 1
            ### implement the same procedure as right sweep ###
            ### append the singular value matrices to the list of Lambdas ###
        
        # === check convergence ===
        if verbose:
            print(h, n, 'dE = ', abs(delta))
        if abs(delta) < atol:
            if verbose:
                print("Converged after {:d} sweeps!".format(n))
                print('Ground-state energy: {:.10f}'.format(energy))
            break
    
    if abs(delta) > atol:
        print("Convergence not reached after  {:d} sweeps!".format(n))
        print(h, n, 'dE = ', abs(delta))

    return energy, Lambdas

 Now, as you obtained the working version of DMRG, you should compare it with the exact diagonalization (ED) data. Consider the $L = 8$ chain with open boundary conditions and validate that the ground state energy at $h/J = 2$ equals -16.88514149.

In [ ]:
run_DMRG(2, 8, 16)[0]